In [1]:
import sys
import os
from pathlib import Path

scripts_dir = Path().resolve()
parent_dir = scripts_dir.parent
sys.path.insert(0, str(parent_dir))

import aeon
import aeon.io.api as api
from aeon.io import reader, video
from aeon.schema.dataset import exp02
from aeon.analysis.utils import visits, distancetravelled

import SSM
from SSM.ssm.plots import gradient_cmap, white_to_color_cmap

import cv2
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, SecondLocator, DateFormatter

import numpy as np
import pandas as pd
import seaborn as sns

import scipy
import pyarrow

import torch

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import pymc3 as pm

import Functions.inference as inference
import Functions.kinematics as kinematics
import Functions.learning as learning

!ls /ceph/aeon

# Long, individual session
root = [Path("/ceph/aeon/aeon/data/raw/AEON2/experiment0.2")]

# Ensure root exists
if not np.all([path.exists() for path in root]):
    print("Cannot find root paths. Check path names or connection.")
    
Exp02Summary = api.load(root, exp02.Metadata).metadata[0].toDict()
Patch_Pos = [device['Regions']['ArrayOfPoint'] for device in Exp02Summary['Devices'] if device['Name'].startswith('ActivityPatch')]

subject_events = api.load(root, exp02.ExperimentalMetadata.SubjectState)
sessions = visits(subject_events[subject_events.id.str.startswith("BAA-")])

short_sessions = sessions.iloc[[4,16,17,20,23,24,25,28,29,30,31]]
long_sessions = sessions.iloc[[8, 10, 11, 14]]

aeon			 aeon_transfer.log  rmhzbm0  test2
aeon_transfer_final.log  db_dumps	    test


In [24]:
display(short_sessions)

,id,weight_enter,weight_exit,enter,exit,duration
4,BAA-1100700,26.3,26.3,2022-03-15 12:40:36.282139778,2022-03-15 15:56:55.801119804,0 days 03:16:19.518980026
16,BAA-1101818,26.2,26.7,2022-06-21 10:27:42.615139961,2022-06-21 13:15:50.564740181,0 days 02:48:07.949600220
17,BAA-1101819,25.4,26.4,2022-06-21 13:28:10.593659878,2022-06-21 16:34:29.241280079,0 days 03:06:18.647620201
20,BAA-1101818,26.4,28.0,2022-06-23 08:39:04.261089801,2022-06-23 11:14:46.121759892,0 days 02:35:41.860670091
23,BAA-1101818,26.9,27.8,2022-06-24 09:32:37.183360100,2022-06-24 12:29:54.365859985,0 days 02:57:17.182499885
24,BAA-1101823,27.9,27.9,2022-07-14 12:53:15.740799904,2022-07-14 16:35:05.846270084,0 days 03:41:50.105470180
25,BAA-1101821,25.0,26.0,2022-07-15 13:14:40.239679813,2022-07-15 15:47:44.568860054,0 days 02:33:04.329180241
28,BAA-1101818,26.7,29.5,2022-08-15 10:25:01.651199818,2022-08-15 13:04:50.849309921,0 days 02:39:49.198110103
29,BAA-1101819,27.5,30.0,2022-08-15 13:11:23.791259766,2022-08-15 16:10:27.350080013,0 days 02:59:03.558820247
30,BAA-1101818,26.5,30.1,2022-08-16 09:25:53.963200092,2022-08-16 12:34:02.296770096,0 days 03:08:08.333570004


In [9]:
display(long_sessions)

,id,weight_enter,weight_exit,enter,exit,duration
8,BAA-1100699,26.75,29.0,2022-03-29 13:11:37.277699947,2022-04-04 09:16:51.086110115,5 days 20:05:13.808410168
10,BAA-1100701,26.70,30.6,2022-04-04 15:54:07.945059776,2022-04-08 16:34:36.183650017,4 days 00:40:28.238590241
11,BAA-1100699,29.00,28.2,2022-04-20 11:51:24.997630119,2022-04-28 10:02:56.655809879,7 days 22:11:31.658179760
14,BAA-1100702,27.00,28.9,2022-05-03 11:08:12.660739899,2022-05-12 09:21:42.411650181,8 days 22:13:29.750910282


In [25]:
for i in range(len(short_sessions)):
    title = 'ShortSession' + str(i)
    print(title)
    try:
        P = np.load('../Data/MouseKinematicParameters/' + title + 'Parameters.npz', allow_pickle=True)
        sigma_a, sigma_x, sigma_y, sqrt_diag_V0_value, B, Qe, m0, V0, Z, R = P['sigma_a'].item(), P['sigma_x'].item(), P['sigma_y'].item(), P['sqrt_diag_V0_value'].item(), P['B'], P['Qe'], P['m0'], P['V0'], P['Z'], P['R']
        print('sigma_a: ', sigma_a)
        print('sigma_x: ', sigma_x)
        print('sigma_y: ', sigma_y)
        print('sqrt_diag_V0: ', sqrt_diag_V0_value)
        
        print(" ")
    except FileNotFoundError: continue

ShortSession0
sigma_a:  214.12889099121094
sigma_x:  -0.5866673798271702
sigma_y:  0.3635427066052031
sqrt_diag_V0:  52.63426146106122
 
ShortSession1
sigma_a:  136.4071044921875
sigma_x:  0.4751876603956538
sigma_y:  0.1930825613531352
sqrt_diag_V0:  53.35174238978601
 
ShortSession2
sigma_a:  504.87884521484375
sigma_x:  0.7613704454108411
sigma_y:  1.6495740120860443
sqrt_diag_V0:  215.6021540192916
 
ShortSession3
sigma_a:  422.48626708984375
sigma_x:  -0.4660510221125924
sigma_y:  0.19663594525980846
sqrt_diag_V0:  152.91396538130388
 
ShortSession4
sigma_a:  232.239501953125
sigma_x:  -0.5222532679091298
sigma_y:  0.30354362596848455
sqrt_diag_V0:  68.18996748692327
 
ShortSession5
sigma_a:  244.88157653808594
sigma_x:  -0.2029959934513377
sigma_y:  -0.7442065242262328
sqrt_diag_V0:  83.86288791764395
 
ShortSession6
sigma_a:  99.7579345703125
sigma_x:  -0.10978131313458078
sigma_y:  0.21133249741003102
sqrt_diag_V0:  32.98934508089078
 
ShortSession7
sigma_a:  290.8110656738281


In [11]:
for i in range(4):
    title = 'LongSession' + str(i)
    print(title)
    try:
        P = np.load('../Data/MouseKinematicParameters/' + title + 'Parameters.npz', allow_pickle=True)
        sigma_a, sigma_x, sigma_y, sqrt_diag_V0_value, B, Qe, m0, V0, Z, R = P['sigma_a'].item(), P['sigma_x'].item(), P['sigma_y'].item(), P['sqrt_diag_V0_value'].item(), P['B'], P['Qe'], P['m0'], P['V0'], P['Z'], P['R']
        print('sigma_a: ', sigma_a)
        print('sqrt_diag_V0: ', sqrt_diag_V0_value)
        print(" ")
    except FileNotFoundError: continue

LongSession0
sigma_a:  236.20263671875
sqrt_diag_V0:  51.061492297298834
 
LongSession1
sigma_a:  240.68421936035156
sqrt_diag_V0:  59.89037970518794
 
LongSession2
sigma_a:  210.2962188720703
sqrt_diag_V0:  48.120707393978876
 
LongSession3
sigma_a:  194.5465087890625
sqrt_diag_V0:  68.78434102918425
 
